In [34]:
import sys, time
import numpy as np
import cv2
import glob
from optparse import OptionParser
from pyntcloud import PyntCloud
import pandas as pd
from IPython.display import IFrame

import matplotlib.pyplot as plt

img = cv2.imread("fullImage.jpg")

if img is None:
    print("Image Not Found at: {}".format(options.image))
    exit()

hgt = img.shape[0]
wth = img.shape[1]
hwth = int(wth/2)

numPx = int(906)
# numPx = int(910)

center = int(hwth/2)

circlePx = int(numPx/2)

In [35]:
start = time.time()
# circlePx = int(wth/2)
uPixel = np.linspace(-center, center, num=hwth, dtype=float)
vPixel = np.linspace(-center, center, num=hgt, dtype=float)

# xx = 2*uPixel/hwth
# yy = 2*vPixel/hgt

diff = center - circlePx
# uPixel = np.linspace(-circlePx, circlePx, num=numPx, dtype=float)
# vPixel = np.linspace(-circlePx, circlePx, num=numPx, dtype=float)

xx = uPixel/circlePx
yy = vPixel/circlePx

print(diff)
print(np.min(xx))
print(np.max(yy))
print(xx.size)

theta = np.zeros((hwth,hwth), dtype = np.float32)
phi = np.zeros((hwth,hwth), dtype = np.float32)
borders = np.zeros((hwth,hwth), dtype = np.float32)

ii = 0
jj = 0
for i, x in enumerate(xx):
    for j, y in enumerate(yy):
        l = x**2+y**2
        if l < 1:
#             print(i,j)
#             ii = i+diff
#             jj = j+diff
            phi[i,j] = np.arctan2(y,x)
            theta[i,j] = np.sqrt(l)*np.pi/2
            borders[i,j] = 1
    if np.mod(i,100) == 0:
        print("Iteration {}".format(i))

plt.imshow(theta)
plt.show()
            
print("ThetaSize: {}".format(theta.shape))
    
print("Iteration Time: {}".format(time.time()-start))

27
-1.0596026490066226
1.0596026490066226
960
Iteration 0
Iteration 100
Iteration 200
Iteration 300
Iteration 400
Iteration 500
Iteration 600
Iteration 700
Iteration 800
Iteration 900
ThetaSize: (960, 960)
Iteration Time: 4.914733648300171


In [36]:
theta_s = np.sin(theta)
theta_c = np.cos(theta)
phi_s = np.sin(phi)
phi_c = np.cos(phi)

x = theta_s*phi_c
y = theta_s*phi_s
z = theta_c

print(np.min(x),np.max(x))
print(np.min(y),np.max(y))
print(np.min(z),np.max(z))

phi_sr = np.sin(phi+np.pi)
phi_cr = np.cos(phi+np.pi)

rx = theta_s*(phi_cr)
ry = theta_s*(phi_sr)

print(x.shape)
print(y.shape)
print(z.shape)
count = np.count_nonzero(borders)
print(count)
# points = np.empty((count,3))
# colours = np.zeros((count,3))

# points = np.empty((hwth*hwth,3))
# rPoints = np.empty((hwth*hwth,3))
# colours = np.zeros((hwth*hwth,3))
# rColours = np.zeros((hwth*hwth,3))

points = np.empty((count,3))
rPoints = np.empty((count,3))
colours = np.zeros((count,3))
rColours = np.zeros((count,3))

k = 0
# for i,t in enumerate(borders!=0):
#     h = ([j for j, h in enumerate(t) if h])
#     for j in h:
#         points[k] = [x[i,j],y[i,j],z[i,j]]
#         k += 1
        
for i in range(hwth):
    for j in range(hwth):
        if borders[i,j] !=0:
            points[k] = [x[i,j],y[i,j],z[i,j]]
            rPoints[k] = [-rx[i,j],ry[i,j],-z[i,j]]
            k += 1

print(points.shape)
print(rPoints.shape)
points = np.concatenate((points,rPoints))
print(points.shape)

-0.9999994 0.9999994
-0.9999994 0.9999994
1.9983449e-05 1.0
(960, 960)
(960, 960)
(960, 960)
643288
(643288, 3)
(643288, 3)
(1286576, 3)


In [37]:
k = 0
for i,t in enumerate(borders!=0):
    h = ([j for j, h in enumerate(t) if h])
    for j in h:
        colours[k] = img[i,j]
        rColours[k] = img[i,j+hwth]
        k += 1

# for i in range(wth):
#     for j in range(wth):
#         colours[k] = img[j,i]
#         k += 1     
        
colours = np.concatenate((colours,rColours))
print(colours.shape)

(1286576, 3)


In [38]:
# derp = np.column_stack((x,y,z))
cPoints = pd.DataFrame(points,columns=['x', 'y', 'z'])
cPoints[['red', 'blue', 'green']] = pd.DataFrame(colours.astype(np.uint8), index=cPoints.index)
cloud = PyntCloud(cPoints)
print("Done")

Done


In [40]:
# cloud.plot(point_size=0.01)